# Ausgewählte Kapitel sozial Webtechnologien - Neuronale Netze
## Trainieren eines Word2Vec Modells und Darstellung von Wort- und Dokumentenvektoren anhand von Anfragetexten des FragDenStaat-Projektes

Bearbeiten von:
* Sebastian Jüngling (558556)
* Konstantin Bruckert (558290)

Prüfer:
* Benjamin Voigt

# Einleitung
Über das FragDenStaat-Portal werden Anfragen an Behörden in Deutschland gesammelt und zur Verfügung gestellt. Durch die stetig wachsende Popularität des Portals liegt diesem Projekt ein umfangreicher Datensatz vor, dessen Informationsgehalt im Laufe dieses Notebooks mithilfe eines Word2Vec Modells möglichst weit ausgeschöpft werden soll.

**Grober Ablauf**<br>
Zunächst werden die bereits bereinigten Daten für die Weiterverarbeitung aufbereitet und randomisiert. Tatsächliche Input-Daten werden daraufhin durch die Indexierung und Speicherung in Lookup-Tables generiert. Mithilfe von Context-Windows können nun Target-Label Paare aus Daten abgeleitet und für alle Sätze erzeugt werden. Das eigentliche Training innerhalb des Notebooks findet dann mithilfe eines Skip-Gram Modells statt, welches in sequenziellen Batches über mehrere Epochen hinweg die Word-Emebddings trainiert. Die daraus resultierenden Word-Embeddings können nun genutzt werden, um Dokumenten-Vektoren aufzubauen und diese oder auch einfache Wort-Vektoren dann mithilfe der Cosine-Similarity zu vergleichen. Abschließend wird versucht, die Word-Embeddings in verschieden Arten und unter Zuhilfenahme des t-SNE Dimensionsreduktionsverfahrens zu visualisieren bzw. Ähnlichkeiten zu clustern.

**Grundlage/Inspiration**<br>
Als Grundlage und grober Leitfaden für das Projekt diente das TensoFlow-Tutorial [Vector Representations of Words](https://www.tensorflow.org/tutorials/representation/word2vec). Ein Großteil der Architekturideen und Implementationsbausteine musste jedoch umgebaut werden und auf die speziellen Bedürfnisse unseres Datensatzes, als auch die Ziele der Projektarbeit angepasst werden.

**Hintergrundinformationen**<br>
Für allgemeine Hintergrundinformationen zu den einzelnen, hier im Modul angewandte Techniken und auch Details zur Entscheidungsfindung für bestimmte Ansätze, lohnt sich zudem ein Blick in das [Exposé](Documents/NN-Projekt-Expose.pdf).

**Quellen**<br>
Quellengestützte Aussagen und direkt/indirekt übernommene Inhalte sind mit einem dreistelligen Tag [XXX] versehen und können anhand diesem im Literaturverzeichnis am Ende des Notebooks eingesehen werden.

In [1]:
# Für Dependency-Installation siehe README.md
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import sys
from tempfile import gettempdir

import numpy as np
from six.moves import xrange
import tensorflow as tf

import pandas as pd

from sklearn.manifold import TSNE

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

%matplotlib inline
import matplotlib.pyplot as plt

# Daten:
Hier findet das Einladen des Anfragen-Katalogs des FragDenStaat-Projektes statt.  
Die Daten wurden schon im Zuge der Werkstudententätigkeit von S. Jüngling im Vorfeld bezogen und weitestgehend aufbereitet.
Dabei wurden die Texte auf ihre bedeutungstragenden Begriffe reduziert und die Wörter lemmatisiert.

Beim Ausführen dieses Notebooks bitte darauf achten, die Daten gemäß der Anleitung in der README.md Datei herunterzuladen.

In [2]:
data = pd.read_json('fds_requests_preprocessed.json', orient='records', encoding='utf-8')
data = data.set_index('id') #set column 'id' as index

In [3]:
data.head()

,description,preprocessed,textrank,title
id,,,,
47033,1. Wann haben die beiden letzten lebensmittelr...,"[[kontrollbericht, parkstern, berlin], [betrie...","[[parkstern, Parkstern, 1.1227777778], [berlin...","Kontrollbericht zu Parkstern, Berlin"
131943,Die Stellungnahme des BfR zur IARC- Monographi...,"[[stellungnahme, bfr], [iarc, monographie, gly...","[[stellungnahme, Stellungnahme, 1.0], [bfr, Bf...",Stellungnahme des BfR zur IARC- Monographie üb...
47827,1. Wann haben die beiden letzten lebensmittelr...,"[[kontrollbericht, aroma, berlin], [betriebsüb...","[[aroma, Aroma, 1.1227777778], [berlin, Berlin...","Kontrollbericht zu Aroma, Berlin"
131938,Die Stellungnahme des BfR zur IARC- Monographi...,"[[stellungnahme, bfr], [iarc, monographie, gly...","[[stellungnahme, Stellungnahme, 1.0], [bfr, Bf...",Stellungnahme des BfR zur IARC- Monographie üb...
48091,1. Wann haben die beiden letzten lebensmittelr...,"[[kontrollbericht], [hans, glück, bonn], [betr...","[[bonn, Bonn, 1.2479166667000001], [hans, Hans...","Kontrollbericht zu ""Hans im Glück"", Bonn"


### Beispiel für Preprocessing des Anfragetextes:

In [4]:
print('Titel und Description für Beispielanfrage:\n')
print(data.loc[47827]['title'])
print(data.loc[47827]['description'])

print('-------------------------------------------------------------------------')

print('\nPreprocessed Anfragetext für Titel und Anfragetext:\n')
print(data.loc[47827]['preprocessed'])

Titel und Description für Beispielanfrage:

Kontrollbericht zu Aroma, Berlin
1. Wann haben die beiden letzten lebensmittelrechtlichen Betriebsüberprüfungen im folgenden Betrieb stattgefunden:
Aroma
Kantstraße
10625 Berlin

2. Kam es hierbei zu Beanstandungen? Falls ja, beantrage ich hiermit die Herausgabe des entsprechenden Kontrollberichts an mich.
-------------------------------------------------------------------------

Preprocessed Anfragetext für Titel und Anfragetext:

[['kontrollbericht', 'aroma', 'berlin'], ['betriebsüberprüfungen', 'betrieb'], ['aroma', 'kantstraße', 'berlin'], ['beanstandung'], ['herausgabe', 'kontrollberichts']]


### Reduzierung der Anzahl der Anfragen zu Glyphosat
Immer wieder kommt es bei FragDenStaat zu einer Häufung tagespolitischer Themen, wie z.B. die Fragen rund um das Thema Glyphosat. Seit März 2019 sind hierzu bereits mehr als 30.000 Anfragen eingegangen, welche anhand eines immer gleichen Musters ausformuliert werden und somit das Training der globalen Datenmenge zu stark beeinflussen. Zur Reduzierung dieses Einflusses werden die Anfragen zu diesem Thema bei 3000 gedeckelt.
Um auch in Zukunft und ggf. bei der Nutzung andersartiger Datensätze einwandfreie Ergebnisse zu erzielen, sollte regelmäßig geprüft werden, ob die Daten von einem bestimmten Thema dominiert werden.

In [5]:
glyphosat_title = 'Stellungnahme des BfR zur IARC- Monographie über Glyphosat'
glyphosat_ids = data[data['title'] == glyphosat_title].index

print('Anzahl der Anfragen zu Glyphosat:', len(glyphosat_ids), 'von insgesamt:', len(data), 'Anfragen')

# Da dies alle gleiche Anfragen sind und diese hohe Anzahl den Trainingsprozess verfälschen würde, 
# wird die Anzahl der Anfragen zu Glyphosat auf 3000 beschränkt
remain_glyphosat_requests = 3000
data.drop(glyphosat_ids[remain_glyphosat_requests:], inplace=True) # drop by id

print('Anzahl der Anfragen zu Glyphosat nach Bereinigung:', len(data[data['title'] == glyphosat_title]), 'von insgesamt:', len(data), 'Anfragen')


Anzahl der Anfragen zu Glyphosat: 36199 von insgesamt: 92374 Anfragen
Anzahl der Anfragen zu Glyphosat nach Bereinigung: 3000 von insgesamt: 59175 Anfragen


### Shuffle Data:
Während des Trainings mit den Daten konnte trotz der sorgfältigen Beseitigung dominanter Themen immer noch eine überproportionale Gewichtung der Glyphosat-Themen festgestellt werden. Das Problem lag hierbei im chronologisch vorliegenden Datensatz, welcher eine natürliche, große Abfolge von gleichen Themen nacheinander besitzt. Um die Word-Embeddings dadurch nicht zu stark in eine Richtung zu trainieren, werden die Anfragen randomisiert.

In [6]:
data = data.sample(frac=1)
data.head()

,description,preprocessed,textrank,title
id,,,,
30037,Sämtliche Einsatzprotokolle der Sicherungsgrup...,"[[einsatzprotokolle, übergriff, kriminalpolize...","[[jugendwohngruppe, Jugendwohngruppe, 1.369075...",Einsatzprotokolle von Übergriff der Berliner K...
41614,1. Wann haben die beiden letzten lebensmittelr...,"[[kontrollbericht, schützenhof, eitorf], [betr...","[[schützenhof, Schützenhof, 1.3868680556], [wi...","Kontrollbericht zu Schützenhof, Eitorf"
9663,ich bitte um eine Übersicht der aktuellen Krip...,"[[kita-gebühren, verbandsgemeinde], [übersicht...","[[verbandsgemeinde, Verbandsgemeinden, 1.24791...",Übersicht über die Kita-Gebühren in den einzel...
2536,In verschiedenen Quellen wird die Weitergabe v...,"[[weitergabe, daten/akten, behörde], [quell, w...","[[behörde, Behörden, 2.8602611613], [informati...",Weitergabe von Daten/Akten an andere Behörden
1333,Bitte übersenden Sie folgende Informationen zu...,"[[information, abendessen, herr, ackermann, ap...","[[abendessen, Abendessen, 2.0533034722], [urte...",Informationen zu Abendessen mit Herrn Ackerman...


# Input-Daten generieren:

## Daten aufbereiten
Für die späteren Bearbeitungsschritte müssen die vorliegenden Anfragen aufbereitet und um Metainformationen erweitert werden.  
Besonders die Indizierung der benutzten Wörter und das Erstellen entsprechender Lookup-Tables spielt im weiteren Verlauf des Notebooks eine vitale Rolle. 
Mit den daraus gewonnenen Wort-Indizes werden die Sätze der Anfragetexte nachgebaut. Für das Training wird außerdem die Gesamtanzahl der einzigartigen Wörter in allen Anfragetexten benötigt.

In [7]:
def build_lookup_tables(docs, vocabulary_size=None):
    '''
    :param docs: Spalte eines pandas-DF: data['preprocessed'].values
    :return sentences: Alle Sätze aller Dokumente in einer Liste
    :return words: Alle Wörter aller Dokumente in einer Liste
    :return word_count: Häufigkeiten der jeweiligen Wörter in allen Dokumenten
    :return word_2_index_dict: Lookup-Table
    :return index_2_word_dict: Lookup-Table
    :return sentences_as_index: Alle Sätze aller Dokumente mit Wortindex, anstatt des Wortes
    :return sentences_as_index_flattened: wie sentences_as_index, aber ohne subarrays (flattened)
    :return vocabulary_size: Anzahl der einzigartigen Wörter
    '''
    sentences = [sent for pd_list in docs for sent in pd_list]
    words = [word for sent in sentences for word in sent]
  
    if not vocabulary_size:
        # Anzahl der einzigartigen Wörter
        vocabulary_size = len(set(words))
 
    # Anzahl der Worthäufigkeiten
    word_count = collections.Counter(words).most_common(vocabulary_size)
    word_count.append(['UNK', -1]) # Flag für Wörter, die nicht oft vorkommen (nur wichtig bei begrenzter Vokabulargröße)
  
    # Lookup-Tables
    word_2_index_dict = {}
    for index, word in enumerate(word_count):
        word_2_index_dict[word[0]] = index
  
    index_2_word_dict = dict(zip(word_2_index_dict.values(), word_2_index_dict.keys()))
  
    # Wörter der Anfragetexte durch Indizes austauschen:
    sentences_as_index = []
    unknown_word_count = 0
    for sent in sentences:
        sent_index = []
        for word in sent:
            if word in word_2_index_dict:
                sent_index.append(word_2_index_dict[word])
            else:
                unknown_word_count += 1
        if sent_index:
            sentences_as_index.append(sent_index)
    word_count[-1][1] = unknown_word_count

    sentences_as_index_flattened = [word for sent in sentences_as_index for word in sent]
  
    return sentences, words, word_count, word_2_index_dict, index_2_word_dict, sentences_as_index, sentences_as_index_flattened, vocabulary_size


In [8]:
sentences, words, word_count, word_2_index_dict, index_2_word_dict, sentences_as_index, sentences_as_index_flattened, vocabulary_size = build_lookup_tables(data['preprocessed'].values)

### Beispiele:

Alle Sätze der Anfragetexte in einer Liste:

In [9]:
sentences[:5]

[['einsatzprotokolle', 'übergriff', 'kriminalpolizei', 'jugendwohngruppe'],
 ['einsatzprotokolle',
  'sicherungsgruppe',
  'kriminalpolizei',
  'mai',
  'jugendwohngruppe',
  'verband',
  'geflüchteter'],
 ['kontrollbericht', 'schützenhof', 'eitorf'],
 ['betriebsüberprüfungen',
  'betrieb',
  'schützenhof',
  'windecker',
  'straße',
  'eitorf'],
 ['beanstandung']]

Alle Wörter aller Anfragetexte chronologisch in einer Liste:

In [10]:
words[:5]

['einsatzprotokolle',
 'übergriff',
 'kriminalpolizei',
 'jugendwohngruppe',
 'einsatzprotokolle']

Anzahl der Worthäufigkeiten:

In [11]:
word_count[:5]

[('betrieb', 24351),
 ('herausgabe', 24007),
 ('kontrollbericht', 23946),
 ('beanstandung', 23880),
 ('betriebsüberprüfungen', 23782)]

Wort zu Wortindex Lookup-Table:

In [12]:
# word_2_index_dict
# nur für Anschauungszwecke (erste 10 Key-Value-Paaare):
{k: word_2_index_dict[k] for k in list(word_2_index_dict)[:10]}

{'betrieb': 0,
 'herausgabe': 1,
 'kontrollbericht': 2,
 'beanstandung': 3,
 'betriebsüberprüfungen': 4,
 'kontrollberichts': 5,
 'dokument': 6,
 'information': 7,
 'abs.': 8,
 'stellungnahme': 9}

Wortindex zu Wort Lookup-Table:

In [13]:
# index_2_word_dict
# nur für Anschauungszwecke (erste 10 Key-Value-Paaare):
{k: index_2_word_dict[k] for k in list(index_2_word_dict)[:10]}

{0: 'betrieb',
 1: 'herausgabe',
 2: 'kontrollbericht',
 3: 'beanstandung',
 4: 'betriebsüberprüfungen',
 5: 'kontrollberichts',
 6: 'dokument',
 7: 'information',
 8: 'abs.',
 9: 'stellungnahme'}

Alle Wörter in allen Anfragesätzen ausgetauscht durch den jeweiligen Wortindex:

In [14]:
sentences_as_index[:5]

[[13598, 2349, 13599, 29978],
 [13598, 53908, 13599, 453, 29978, 272, 3883],
 [2, 13600, 10080],
 [4, 0, 13600, 53909, 24, 10080],
 [3]]

Alle Wörter aller Anfragetexte chronologisch in einer Liste, ausgetauscht durch den Wortindex:

In [15]:
sentences_as_index_flattened[:10]

[13598, 2349, 13599, 29978, 13598, 53908, 13599, 453, 29978, 272]

Anzahl aller einzigartigen Wörter aus allen Anfragetexten:

In [16]:
vocabulary_size

95602

## (Input-)Target-Wörter mit entsprechenden Labels aus Daten ableiten


### Word-Embeddings
Essenziell für die folgenden Schritte ist ein generelles Verständnis von Word-Embeddings. Ein Wort kann als ein Vektor mit einer beliebigen Anzahl von Features dargestellt werden. Die voreingestellten Parameter dieses Notebooks arbeiten mit 300 Features (Vektordimension).
Vergleichen wir beispielsweise die Word-Embeddings der Wörter "Hund" und "Katze", so werden bestimmte Features innerhalb der beiden Vektoren eine Ähnlichkeit haben, u.a. an der Stelle wo das Modell die Kategorie "Tier" trainiert hat. Anhand eines Beispiels, in dem die Word-Embedding Values mit Farben je nach Wert ersetzt wurden, lässt sich dieses Prinzip gut veranschaulichen:

<img src="Images/king-man-woman-embedding.png" alt="drawing" width="500"/>
Quelle: [WED]


### Language Modelling
Je nach Ansatz ist es das Ziel eines Language Modells, für ein gegebenes Wort möglichst Präzise Vorhersagen zu treffen, welches Wort darauf folgen könnte (CBOW) oder anhand eines Wortes die umgebenden Wörter vorherzusagen (Skip-Gram). Anhand der im vorherigen Abschnitt erstellten Word-Embeddings können wir einzelne Wörter oder ganze Sätze leicht vergleichen und prüfen, ob Sie in einem kontextuellen Zusammenhang stehen. 

### Skip-Gram Model und Context-Windows
Folgende Abbildung skizziert die generelle Architektur eines Skip-Gram Modells:

<img src="Images/skip_gram_net_architecuture.png" alt="drawing" width="600"/>
Quelle: [SAM]

Im Notebook kommt es jedoch zu einigen Abweichungen von der Standard-Architektur, insbesondere bei der Klassifikation/Loss-Funktion (siehe folgende Kapitel: NCE). <br>

Natürlich muss ein solches Language-Modell zunächst ausgiebig trainiert werden. Mittels Context-Windows, also ein Ausschnitt von umgebenden Wörtern, versuchen wir Wort-Paare aus Target- und Label-Wörtern zu erstellen, die häufig zusammen auftreten. Angenommen wir nutzen Window-Size=2, dann betrachten wir an jedem Wort eines Satzes die zwei Wörter ("labels") vor und nach dem fokussierten Wort ("target") und notieren dieses gemeinsame Auftreten. In folgendem Beispiel wird die Context-Window Methode an einem Skip-Gram Modell dargestellt. Blau markierte Target-Wörter (Input) mit den entsprechenden Labels, abhängig von der Window-Size, gespeichert als Target-Label Paare (Tupel).

<img src="Images/skipgram-sliding-window-samples.png" alt="drawing" width="400"/>
Quelle: [SWS]

Context-Windows arbeiten mit einzelnen Sätzen und beim Beginn und Ende eines jeden Satzes muss zudem darauf geachtet werden, dass mit dem Window nicht vor oder nach dem Satz geslided wird. Die folgende Implementierung erzeugt nun die Target-Label Paare, wobei anstelle realer Wörter im weiteren Verlauf die Wort-Indizes verwendet werden (siehe Beispiel). 

In [17]:
def get_context_window(input_data, target_index, window_size):
    '''
    Ermittelt umgebene Wörter abhängig von window_size und target_index des Wortes
    :param input_data: Liste mit Wortindizes
    :param: target_index: Listenindex des jeweiligen Wortes
    :param window_size: Anzahl der Wörter links und rechts des target wortes
    :return target value und liste der umgebenen Wörter
    '''
  
    left_start_index = target_index - window_size if (target_index - window_size) >= 0 else 0
  
    right_start_index = target_index + 1
    right_stop_index = target_index + window_size + 1
  
    target = input_data[target_index]
    left_window = input_data[left_start_index:target_index]
    right_window = input_data[right_start_index:right_stop_index]
  
    return target, left_window + right_window
   

Beispiel:

In [18]:
input_data = list(range(6))
print('Beispielsatz aus Indizes:', input_data)
print()

target_word_index, context_words = get_context_window(input_data, 0, 2)
print('Target-Wort für Index:', target_word_index)
print('Context-Wörter für window_size 2:', context_words)

target_word_index, context_words = get_context_window(input_data, 2, 2)
print('\nTarget-Wort für Index::', target_word_index)
print('Context-Wörter für window_size 2:', context_words)

Beispielsatz aus Indizes: [0, 1, 2, 3, 4, 5]

Target-Wort für Index: 0
Context-Wörter für window_size 2: [1, 2]

Target-Wort für Index:: 2
Context-Wörter für window_size 2: [0, 1, 3, 4]


### Generiere Target-Wortliste mit entsprechenden Wortkontexten für alle Sätze der Anfragetexte:
Nach den Beispielen kann die Context-Window Funktion nun auf alle Sätze in den Input-Daten angewandt werden. Weitere Beispiele mit Ausschnitten aus dem Gesamtdatenbestand bieten zudem zur besseren Veranschaulichung ein Re-Mapping der Word-Indexe auf die realen Wörter.

In [19]:
def build_targets_and_labels(input_data, window_size):
    '''
    Ermittelt alle Targets und Labels abhängig von der window_size
    :param input_data: Liste mit Sublisten (Sätze)
    :param window_size: Anzahl der Wörter links und rechts des target wortes
    :return targets: Liste aller Targetwörter
    :return labels: Liste aller Labels zu jeweiligem Targetwort
    '''
    targets = []
    labels = []
    for sent in input_data:
        for index, word in enumerate(sent):
            target_word_index, context_words = get_context_window(sent, index, window_size)
            for context_word in context_words:
                targets.append(target_word_index)
                labels.append(context_word)
    return targets, labels
  

Beispiel:

In [20]:
input_data = sentences_as_index[:3]
print('Die ersten drei Beispielsätze mit Wortinidizes:\n', input_data, '\n')

targets_list, labels_list = build_targets_and_labels(input_data, window_size=2)
print('Target Wortindizes: \n', targets_list)
print('Label Wortindizes: \n', labels_list)

print('\nBeispiel für Mapping der Wortinindizes der Target und Label-Liste:')
for i in range(len(targets_list)):
    print(targets_list[i], index_2_word_dict[targets_list[i]], '->', labels_list[i], index_2_word_dict[labels_list[i]])
    
    

Die ersten drei Beispielsätze mit Wortinidizes:
 [[13598, 2349, 13599, 29978], [13598, 53908, 13599, 453, 29978, 272, 3883], [2, 13600, 10080]] 

Target Wortindizes: 
 [13598, 13598, 2349, 2349, 2349, 13599, 13599, 13599, 29978, 29978, 13598, 13598, 53908, 53908, 53908, 13599, 13599, 13599, 13599, 453, 453, 453, 453, 29978, 29978, 29978, 29978, 272, 272, 272, 3883, 3883, 2, 2, 13600, 13600, 10080, 10080]
Label Wortindizes: 
 [2349, 13599, 13598, 13599, 29978, 13598, 2349, 29978, 2349, 13599, 53908, 13599, 13598, 13599, 453, 13598, 53908, 453, 29978, 53908, 13599, 29978, 272, 13599, 453, 272, 3883, 453, 29978, 3883, 29978, 272, 13600, 10080, 2, 10080, 2, 13600]

Beispiel für Mapping der Wortinindizes der Target und Label-Liste:
13598 einsatzprotokolle -> 2349 übergriff
13598 einsatzprotokolle -> 13599 kriminalpolizei
2349 übergriff -> 13598 einsatzprotokolle
2349 übergriff -> 13599 kriminalpolizei
2349 übergriff -> 29978 jugendwohngruppe
13599 kriminalpolizei -> 13598 einsatzprotokolle


# Generiere Trainings-Batch:
Folgende Hilfsfunktion generiert iterativ Teilsequenzen (Batches) aus den Target-Label Paaren. Im Return der Funktion ist ein einzelner Batch und mithilfe der Variable `data_index` wird die aktuelle Iterationsposition im Gesamtdatensatz zwischengespeichert und resetted, sobald das Ende des Gesamtdatensatzes erreicht wurde.

In [21]:
data_index = 0
def generate_batch(batch_size, targets_list, labels_list):
    '''
    Generiert den Trainigs-Batch
    :return batch: Liste mit Target-Indizes
    :return labels: Liste mit Label-Indizes, zugehörig zu Target an gleicher Position in batch
    '''
    global data_index
    if data_index + batch_size > len(targets_list):
        data_index = 0
    batch = np.array(targets_list[data_index:data_index + batch_size], dtype=np.int32)
    labels = np.array(labels_list[data_index:data_index + batch_size], dtype=np.int32)[:, np.newaxis]
    data_index += batch_size
  
    return batch, labels
  

Beispiel für batch_size = 8:

In [22]:
batch, labels = generate_batch(8, targets_list, labels_list)
print('Target Indizes:')
print(batch)

print('\nLabel Indizes')
print(labels)

print('\nBeispiel für Mapping der Wortinindizes der Target und Label-liste:')
for i in range(len(batch)):
    print(batch[i], index_2_word_dict[batch[i]], '->', labels[i, 0], index_2_word_dict[labels[i, 0]])

Target Indizes:
[13598 13598  2349  2349  2349 13599 13599 13599]

Label Indizes
[[ 2349]
 [13599]
 [13598]
 [13599]
 [29978]
 [13598]
 [ 2349]
 [29978]]

Beispiel für Mapping der Wortinindizes der Target und Label-liste:
13598 einsatzprotokolle -> 2349 übergriff
13598 einsatzprotokolle -> 13599 kriminalpolizei
2349 übergriff -> 13598 einsatzprotokolle
2349 übergriff -> 13599 kriminalpolizei
2349 übergriff -> 29978 jugendwohngruppe
13599 kriminalpolizei -> 13598 einsatzprotokolle
13599 kriminalpolizei -> 2349 übergriff
13599 kriminalpolizei -> 29978 jugendwohngruppe


# Baue und Trainiere das Skip-Gram Modell
Bis hierhin sollte klar sein was die Funktion eines Skip-Gram Models und deren Target-Label Paaren, eines Context-Window und eines Batches ist. Mit diesem Wissen steht dem eigentlichen Model-Training nichts mehr im Wege! Fast. Vor dem Training bietet sich zunächst die letzte Möglichkeit, mit einstellbaren Parametern Einfluss auf den Verlauf des Trainings zu nehmen. Neben den bereits bekannten Begriffen müssen noch ein paar wenige, neue Parameter verstanden und eingestellt werden, die einen erheblichen Einfluss auf den weiteren Trainingsverlauf ausüben.

### Einstellbare Parameter:

* `batch_size`: Anzahl der berücksichtigten Target-Label Paare pro Trainingsiteration
* `embedding_size`: Dimensionsgröße der Word-Embeddings 
    * -> Wert von 300 zeigt sich als guter Kompromiss von Performance und Genauigkeit
* `window_size`: Anzahl der berüchtigten Wörter links und rechts (Nachbarwörter) eines Targetwortes für Bildung der Target-Label Paare
* `negative_samples` Anzahl der Negative Samples für NCE Loss (siehe weiter unten)
* Anpassung der sich exponentiell verringerten Lernrate (Start-/End-Learning-Rate)
* `epochs`: Anzahl der Epochen, also Anzahl der Iterationen die benötigt werden, um einmal alle Target-Label Paare zu durchlaufen

In [23]:
batch_size = 128 
embedding_size = 300 # Dimension der Word-Embeddings
window_size = 2  # Anzahl der Wörter links und rechts vom Target-Wort
negative_samples = 64 # Anzahl der negative samples

# Die Lernrate wird je Iteration verringert:
starter_learning_rate = 1.0
end_learning_rate = 0.1

epochs = 6 # Anzahl der Epochen. Eine Epoche ist eine Iteration durch den kompletten Trainingsbestand
print_every_x_step = 2000 # alle x Iterationen werden infos geprintet

## Generieren der Target- und Labelliste für alle Anfragetexte, basierend auf der eingestellten Window-Size:

Zu Beginn des Trainings werden nun alle Target-Label Paare aller Anfragetexte des Gesamtdatenbestandes aufgrund der im vorherigen Schritt eingestellten Window-Size generiert.

In [24]:
targets_list, labels_list = build_targets_and_labels(sentences_as_index, window_size=window_size)

Es folgen einige Berechnungen für wichtige, fixe Parameter, die aus den einstellbaren Parametern abgeleitet werden können.

In [25]:
data_index = 0 # für batch start

exp_decay_lr = starter_learning_rate - end_learning_rate

input_length = len(targets_list) # Anzahl der Target-Label-Paare
print('Anzahl Target-Label-Paare:', input_length)

full_iteration_cycle = int(math.ceil(input_length / batch_size))
print(full_iteration_cycle, 'Iterationen werden benötigt um einmal während des Trainings durch alle Target-Label Paare zu iterieren')

num_steps = full_iteration_cycle * epochs
print('Iterationen während des Trainings:', num_steps)

epsilon=1e-12 # dont touch -> vermeidet Division mit Null bei Normalisierung


Anzahl Target-Label-Paare: 3099230
24213 Iterationen werden benötigt um einmal während des Trainings durch alle Target-Label Paare zu iterieren
Iterationen während des Trainings: 145278


# Build Graph:

Mit eingestellten Parametern und allen generierten Target-Label Paaren der Anfragetexte kann nun die Modellarchitektur festgelegt werden. Zunächst werden Tensorflow Placeholder definiert und dann die Word-Embedings (Anzahl einzigartiger Wörter x Anzahl Features/Dimension) mit einem random Wert zwischen 0 und 1 initialisiert. Auf die gleiche Weiße erhalten auch unsere NCE-Weights (Erklärung folgt) initiale Werte. Der Bias wird mit Nullen initialisiert. Bereits im vorherigen Schritt wurde die Start-Lernrate errechnet (delta aus maximal- und minimal-Lernrate), welche nun in der Modell-Architektur in eine exponential-decay Funktion gepackt wird und somit im späteren Training evolutionär angepasst werden kann. Für die komplette Trainingsarchitektur fehlt jetzt nur noch ein(e) Klassifizierungsverfahren/Loss-Funktion.

## Noise Contrastive estimation (NCE)

Remember: Für ein gegebenes Wort (Target) soll das Modell die Wahrscheinlichkeit der passenden Kontextwörter (Labels) bestimmen. Ein weitverbreitetes und erfahrungsgemäß gutes Klassifizierungsverfahren hierfür ist der Softmax-Classifier. Dieser stellte sich aber während des Trainings als maximal rechenintensiv heraus, da für jedes Wort das ganze Vokabular (mehr als 1 Mio Wörter) mit Wahrscheinlichkeiten belegt werden muss, bzw. die Gewichtsmatrix `[embedding_dim x vocab_dim]` sehr groß ist ([Wiederholung: Softmax-Klassifier](https://www.pyimagesearch.com/2016/09/12/softmax-classifiers-explained/), [Hintergrundwissen: Warum kein Softmax?](https://arxiv.org/pdf/1410.8251.pdf))

Um diese Rechenoperation zu vermeiden, aber trotzdem keine spürbaren Einbußen beim Training der Word-Emebdings zu erleiden, kann der NCE-Loss genutzt werden. Hierbei wird das Gesamtvokabular, also die Menge an Wörtern die auf Ähnlichkeit überprüft werden, auf einen Teil mit dem/den passenden Label/n und einen Teil mit gänzlich unpassenden Wörtern ("Contrastive Noise") reduziert. Zudem wird die Vorhersage auf ein Klassifikationsproblem (richtig/falsch) reduziert (vgl. Softmax: Normalverteilt), (korrekte) Labels und (gegensätzliche) Noise bekommen also die Werte 0 und 1. Das Modell kann somit anhand sehr guter und schlechter Begriffe lernen, ohne dabei jedes Mal das gesamte Vokabular durchforsten zu müssen. Hierin liegt zugleich der entscheidende Punkt bei NCE: Wir können unsere Noise-Verteilung frei bestimmen, z.B. können nur Wörter mit ganz geringer Häufigkeit, alle Wörter mit gleicher Wahrscheinlichkeit oder einfach eine Normalverteilung selektiert werden. Durch diese künstliche Reduzierung der Grunddatenmenge wird jedoch nicht die Genauigkeit des Models in Mitleidenschaft gezogen (QUELLE: [NCE]). 
Da das Problem bereits auf eine klassische "Richtig oder Falsch"-Frage reduziert wurde, kann in NCE die logistische Regression zur Klassifikation herangezogen werden und dann unsere Word-Embeddings mit dem GradientDescentOptimizer optimiert werden. Eine weiterführende Erklärung mit detaillierten Hintergrundinfos zu NCE findet sich [hier](https://towardsdatascience.com/noise-contrastive-estimation-246446ea9aba) und [hier](http://demo.clab.cs.cmu.edu/cdyer/nce_notes.pdf).

In einer früheren Version des Notebooks wurde in diesem Abschnitt auch noch eine Normalisierung der Word-Embeddings auf Werte zwischen 0 und 1 durchgeführt. In zahlreichen Testläufen konnte sich dadurch jedoch keine verbesserte Genauigkeit oder Performance ausmachen und die Ergebnisse der Cosine-Similarity wurden verfälscht, weswegen dieser Part auskommentiert ist.


In [44]:
graph = tf.Graph()

with graph.as_default():
    # Input Daten
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        iteration = tf.placeholder(tf.int32)
  
    # Benutze CPU:
    with tf.device('/cpu:0'):
        with tf.name_scope('embeddings'):
            #embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            # Initialisiere Embeddings:
            embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], 0.001, 1.0))
            # Embeddings Lookup-Table:
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
     
        # Initialisiere NCE-Weights für den NCE-Loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal(
                    [vocabulary_size, embedding_size],
                    stddev=1.0 / math.sqrt(embedding_size)
                )
            )
        
        # Bias:
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Berechnet der durchschnittlichen NCE-Loss pro Batch
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=negative_samples,
                num_classes=vocabulary_size)
        )      
  
    # Lernrate:
    with tf.name_scope('lr'):
        # Verringert Lernrate exponentiell, abhäging von der aktuellen Trainigsiteration
        lr = end_learning_rate +  tf.train.exponential_decay(exp_decay_lr, iteration, 10000, 1/math.e)

    # Gradient Descent optimizer mit entsprechender Lernrate und Minimieren des Loss
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
    # Normalisierung der Embeddings in Range [0, 1] --> Disabled
    '''normalized_embeddings = tf.math.divide(
        tf.subtract(
            embeddings,
            tf.reduce_min(embeddings)
        ),
        tf.maximum(
            tf.subtract(
                tf.reduce_max(embeddings),
                tf.reduce_min(embeddings)
            ),
            epsilon
        )
    )

    #normalized_embeddings = tf.to_float(normalized_embeddings)
    normalized_embeddings = tf.dtypes.cast(normalized_embeddings, tf.float32)'''
 
    init = tf.global_variables_initializer()

# Training: 
Alle Vorbereitungsschritte des Notebooks werden hier zusammengeführt. Das Modell wird mit den eingestellten Parametern und den definierten Input-Daten trainiert, die Zwischenstände werden ausgegeben und die finalen Word-Embeddings werden gespeichert.

In [27]:
with tf.Session(graph=graph) as session:
    # Initialisieren alle Variablen
    init.run()
  
    average_loss = 0
  
    # Training:
    for step in xrange(num_steps):
        # generieren der Batches
        batch_inputs, batch_labels = generate_batch(batch_size, targets_list, labels_list)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels, iteration: step}
    
        # Ausführen eines einzelnen Update Steps, durch Evaluierung des GradientDescent Optimizers
        # Minimieren des Loss und Updaten der Gewichte
        _, loss_val, learn_rate = session.run(
            [optimizer, loss, lr],
            feed_dict=feed_dict)
    
        average_loss += loss_val
        
        # print Lernrate
        if step % print_every_x_step == 0: 
            if step > 0:
                average_loss /= print_every_x_step
            print('Iteration-Step:', step)
            print('\tAverage loss:\t', average_loss, '\n\tlearning-rate:\t', learn_rate)
            average_loss = 0
        
    # Generierte Embeddings für weitere Schritte verfügbar machen:
    #final_embeddings = normalized_embeddings.eval() # --> Disabled
    final_embeddings = embeddings.eval()
    

Iteration-Step: 0
	Average loss:	 315.8806457519531 
	learning-rate:	 1.0
Iteration-Step: 2000
	Average loss:	 147.57861945724488 
	learning-rate:	 0.8368577
Iteration-Step: 4000
	Average loss:	 82.67597859477996 
	learning-rate:	 0.7032881
Iteration-Step: 6000
	Average loss:	 60.39438907146454 
	learning-rate:	 0.5939305
Iteration-Step: 8000
	Average loss:	 46.222604497790336 
	learning-rate:	 0.5043961
Iteration-Step: 10000
	Average loss:	 36.790105431139466 
	learning-rate:	 0.4310915
Iteration-Step: 12000
	Average loss:	 31.503256531476975 
	learning-rate:	 0.37107483
Iteration-Step: 14000
	Average loss:	 26.07833284395933 
	learning-rate:	 0.32193726
Iteration-Step: 16000
	Average loss:	 22.53845557963848 
	learning-rate:	 0.2817069
Iteration-Step: 18000
	Average loss:	 20.246237768888474 
	learning-rate:	 0.24876902
Iteration-Step: 20000
	Average loss:	 18.25406939935684 
	learning-rate:	 0.22180176
Iteration-Step: 22000
	Average loss:	 16.7635548517704 
	learning-rate:	 0.199722

# Dokumenten-Vektoren generieren:
Die im vorherigen Schritt trainierten Word-Embeddings können nun genutzt werden, um pro Anfragetext einen Dokumenten-Vektor aus den Word-Embeddings der jeweiligen Wörter des Textes abzuleiten.
Dabei werden alle Word-Embeddings eines Dokuments spaltenweise gemittelt, um dieses Dokument dann anhand eines einzelnen Vektors zu repräsentieren.

In [28]:
def get_doc_embedding(doc):
    '''
    Generiert Dokumenten-Vektor für ein Dokument, abhängig von den zugehörigen Word-Embeddings
    :param doc: Liste aller Sätze mit preprocessed Words
    :return: Dokumenten-Vektor
    '''
    word_vecs = np.array([
        final_embeddings[word_2_index_dict[word]] for sent in doc for word in sent if word in word_2_index_dict
    ])
    return np.mean(word_vecs, axis=0)

### Dokumenten-Vektoren für alle Anfragen generieren:

In [29]:
data['doc_vector'] = data['preprocessed'].apply(lambda prepr_text: get_doc_embedding(prepr_text) if prepr_text else np.nan)


Aufgrund vereinzelter Fehler im Preprocessing entstehen einige Null-Rows die hier manuell korriegiert werden müssen:

In [30]:
data.isnull().sum()

description     0
preprocessed    0
textrank        0
title           0
doc_vector      7
dtype: int64

In [31]:
data.dropna(inplace=True)

In [32]:
data.isnull().sum()

description     0
preprocessed    0
textrank        0
title           0
doc_vector      0
dtype: int64

Auszug aus den Daten mit Dokumenten-Vektoren:

In [33]:
data.head()

,description,preprocessed,textrank,title,doc_vector
id,,,,,
30037,Sämtliche Einsatzprotokolle der Sicherungsgrup...,"[[einsatzprotokolle, übergriff, kriminalpolize...","[[jugendwohngruppe, Jugendwohngruppe, 1.369075...",Einsatzprotokolle von Übergriff der Berliner K...,"[0.5323159, 0.5101424, 0.38201094, 0.39275265,..."
41614,1. Wann haben die beiden letzten lebensmittelr...,"[[kontrollbericht, schützenhof, eitorf], [betr...","[[schützenhof, Schützenhof, 1.3868680556], [wi...","Kontrollbericht zu Schützenhof, Eitorf","[0.4522284, 0.737864, -0.09975869, 0.73455733,..."
9663,ich bitte um eine Übersicht der aktuellen Krip...,"[[kita-gebühren, verbandsgemeinde], [übersicht...","[[verbandsgemeinde, Verbandsgemeinden, 1.24791...",Übersicht über die Kita-Gebühren in den einzel...,"[0.24316639, 0.5112184, 0.020094834, 0.3383331..."
2536,In verschiedenen Quellen wird die Weitergabe v...,"[[weitergabe, daten/akten, behörde], [quell, w...","[[behörde, Behörden, 2.8602611613], [informati...",Weitergabe von Daten/Akten an andere Behörden,"[0.25974184, 0.44923338, 0.17856327, 0.3782094..."
1333,Bitte übersenden Sie folgende Informationen zu...,"[[information, abendessen, herr, ackermann, ap...","[[abendessen, Abendessen, 2.0533034722], [urte...",Informationen zu Abendessen mit Herrn Ackerman...,"[0.30172154, 0.40615717, 0.1527618, 0.26614517..."


# Cosine Similarity:
Cosine Similarity ist eine gängige Möglichkeit, um die Ähnlichkeit zweier Vektoren zu ermitteln. Mithilfe des Abstandswinkels zwischen den beiden Vektoren wird ein Wert zwischen 0 und 1 erzeugt, welcher das Ähnlichkeitsmaß ausdrückt. Einfache, 2-Dimensionale Vektoren können hierbei noch per Hand errechnet werden. Cosine-Similarity funktioniert jedoch mit einer beliebigen Anzahl von Vektor-Dimensionen und ist somit für den Vergleich unserer Dokumenten- oder Wort-Vektoren hervorragend geeignet.

Formel:
![image](Images/Cosine_Similarity_Formula.png)


Quelle: [CSM]

Beispiel:

|Person/Eigenschaft|  EG1 	| EG2 	|  EG3 	|
|:-----------:	|:----:	|:---:	|:----:	|
|  Konstantin 	| 10 	| 50 	|  200 	|
|  Sebastian  	|  400 	| 100 	| 20 	|
| Prof. Herta 	|  10 	| 5 	|  1 	|

Ähnlichkeit von Konstantin und Sebastian: <br>
Cosine_Similarity ( [10,50,200] , [400, 100, 20]) = 0.15

Ähnlichkeit von Sebastian und Prof. Herta: <br>
Cosine_Similarity ( [400, 100, 20], [10, 5, 1]) = 0.23

Ähnlichkeit von Konstantin und Prof. Herta: <br>
Cosine_Similarity ( [10,50,200], [10, 5, 1]) = 0.77

In [34]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

Negativbeispiel für Vergleich unterschiedlicher Wörter:

In [35]:
word1 = 'herausgabe'
word2 = 'bekleidung'
print('cosine similarity for', word1, 'and', word2)
print('\t', similarity(final_embeddings[word_2_index_dict[word1]], final_embeddings[word_2_index_dict[word2]]))

cosine similarity for herausgabe and bekleidung
	 0.42096847


Positivbeispiel für Vergleich ähnlicher Wörter:

In [36]:
word1 = 'bfr'
word2 = 'iarc' # beide Kürzel stehen in Zusammenhang durch Verwendung in Glyphosatberichten
print('cosine similarity for', word1, 'and', word2)
print('\t', similarity(final_embeddings[word_2_index_dict[word1]], final_embeddings[word_2_index_dict[word2]]))

cosine similarity for bfr and iarc
	 0.74131197


#### Vergleich von Dokumentenvektoren:

In [37]:
def doc_similarity(df, id_1, id_2):
    print('Doc #1:')
    print('\tTitel:', df.loc[id_1]['title'])
    
    print('\nDoc #2:')
    print('\tTitel:', df.loc[id_2]['title'])
    
    print('\nSimilarity [0-1]:', similarity(df.loc[id_1]['doc_vector'], df.loc[id_2]['doc_vector']))

Positivbeispiel für Vergleich ähnlicher Dokumente:

In [38]:
doc_similarity(data, 58945, 48729)

Doc #1:
	Titel: Kontrollbericht zu Bartz, Arzfeld

Doc #2:
	Titel: Kontrollbericht zu Mamma Italia, Esslingen am Neckar

Similarity [0-1]: 0.9522405


Negativbeispiel für Vergleich unterschiedlicher Dokumente:

In [39]:
doc_similarity(data, 58945, 58948)

Doc #1:
	Titel: Kontrollbericht zu Bartz, Arzfeld

Doc #2:
	Titel: Ortsumgehung Barnstorf, Landreis Diepholz

Similarity [0-1]: 0.79113513


## t-SNE

Mit den errechneten Document-Embeddings erhalten wir eine hochdimensionale (in unserem Fall 300) und komplexe Datenmatrix, die sich dank des t-SNE Algorithmus in geringeren Dimensionen, meist 2- oder 3-dimensional, visualisieren lässt. Natürlich erzeugt diese Verringerung einen gewissen Informationsverlust, der wie im folgendem Beschrieben möglichst gering gehalten werden soll. Eine Darstellung unserer Document-Embeddings mittels t-SNE ist besonders geeignet um Strukturen im visualisierten Datensatz, wie lokale Nachbarschaften oder Wort-/Dokumenten-Gruppierung, zu erkennen. Ähnliche Anfragen werden sich somit voraussichtlich clustern und es entsteht eine dreidimensionale Visualisierung, die einen ersten Überblick über die semantischen Zusammenhänge zwischen den Anfragetexten gibt.

## t-SNE Grundprinzip:
Wie so oft, muss das Rad nicht neu erfunden werden und man kann eine optimierte Implementierung des t-SNE bereits über das sklearn-kit beziehen. Um zu verstehen, was sich im Hintergrund abspielt, hier ein Versuch die Funktionsweise anhand eines Beispiels zu erläutern:

**Das Ziel**: Punkte aus dem 2-dimensionalen Raum, 1-dimensional darzustellen.

<img src="Images/tSNE_Ziel.png" alt="drawing" width="300"/>
Quelle: [TSN]

**Abstände Messen:** Dazu wird ein Punkt in den Fokus genommen und die Abstände zu den anderen Punkten wird normalverteilt gemessen. Aus der höheren Dimension wird also die Entfernung auf der reduzierten Dimension abgebildet und dann die Entfernung zur Normalverteilungskurve gemessen.

<img src="Images/tSNE_Normalverteilt.png" alt="drawing" width="300"/>
Quelle: [TSN]

**Normalverteilung in Matrix überführen:** Dadurch ist es möglich, eine Matrix aufzubauen, wo zu jedem Punkt die Entfernung zu einem anderen Punkt (bzw. dessen Entfernung zur Normalverteilungskurve) abgebildet werden kann. Eine Entfernung von einem Punkt zu sich selbst macht hierbei natürlich keinen Sinn.

<img src="Images/tSNE_Matrix1.png" alt="drawing" width="300"/>
Quelle: [TSN]

**Wiederholung mit t-Verteilung:** Im nächsten Schritt wird der gesamte Prozess bis hierhin wiederholt, jedoch nicht mit der t-Verteilung und einem Freiheitsgrad (Cauchy-Verteilung) anstelle der Normalverteilung. Es entsteht eine weit unpräzisere Anordnung der Punkte in unserer t-Matrix:

<img src="Images/tSNE_Matrix2.png" alt="drawing" width="300"/>
Quelle: [TSN]

Auch auf die x-Achse projiziert ergeben die Entfernungen (noch) keinen Sinn:

<img src="Images/tSNE_t_map.png" alt="drawing" width="300"/>
Quelle: [TSN]

**t-Verteilung und Normalverteilung angleichen:** Im nächsten Schritt versuchen wir, die Punkte in der niedrigeren Dimension (hier: 1-D, bzw. x-Achse) der t-Verteilung so lange zu verschieben, bis dieser denen, der Normalverteilung gleichen. Dazu schiebt der t-SNE Algorithmus immer einen Punkt in der Matrix näher an seine Ziel-Koordinate und vollzieht den gleichen Schritt auf der x-Achse.

<img src="Images/tSNE_t_finish.png" alt="drawing" width="300"/>
Quelle: [TSN]

**Weiterer Verlauf:** Im Beispiel wurde von 2- auf 1-Dimensionalität reduziert. Voreingestellte Parameter im Notebook trainieren jedoch Word-Embeddings mit deutlich mehr Dimensionen (hier: 300). Schrittweise reduziert t-SNE die Dimensionen, eine nach der anderen, bis zur gewünschten Ziel-Dimension. Für normale Bildschirme wären das die Dimensionen 1-3 (Beispiel siehe folgende Kapitel).


### Warum wird die t-Verteilung hier benötigt? 
Hier wird es kompliziert: Das zugrunde liegende Verfahren erfordert ein tiefer gehendes mathematisches Verständnis von Verteilungsfunktionen und sprengt den Rahmen des Notebooks. Hilfreiche Links und Sekundärliteratur finden sich [hier](https://www.oreilly.com/learning/an-illustrated-introduction-to-the-t-sne-algorithm) und [hier](http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). 
<br><br>Eine kurze Erklärung in eigenen Worten:
Der Ansatz für die Nutzung der t-Verteilung liegt im Unterschied der Verteilungsfunktionen. 
Würde beim Reduzieren der Dimensionen eines Datensatzes die Normalverteilung herangezogen werden, so käme es zu einer Disbalance der Verteilung von Punkten und deren Distanz zu Ihren Nachbarn. Der Grund hierfür ist der große Unterschied des Maßes einer Entfernung in unterschiedlichen Dimensionen, welche der Algorithmus versucht abzubilden. Um dieser Divergenz an Maßen gerecht zu werden, nutzt t-SNE die t-Verteilung (daher auch das "t"), welche in den Außenbereichen einen viel geringeren Abstieg hat und generell flacher und nicht so intensiv gipfelnd ist wie die Normalverteilung. Konkret bedeutet das: Trotzdem zwei Punkte in der Ausgangsdimension sehr weit voneinander entfernt sind, können diese in der niedrigeren Dimension eine Ähnlichkeit aufweisen.
Quelle: [TDS]

### Hyperparameter

Neben den Input-Daten, der Zieldimension und der Lern-Rate gibt es zwei entscheidende Parameter, die helfen, das meiste aus t-SNE herauszuholen: Perplexity und Iterationsanzahl.

Beim Tunen der **Anzahl an Iterationen** konnten auf den FragDenStaat Datensatz beste Ergebnisse bei ca. 1000 Iterationen festgestellt werden. Diese Zahl stellt jedoch keine fixe Empfehlung dar und muss je nach Grunddatenmenge und Struktur des Datensatzes ausprobiert und angepasst werden. Am Beispiel in der folgenden Grafik lässt sich demonstrativ erkennen, dass bei zu wenigen Iterationen eine schlechte, punktlastige Gruppierung erfolgt. Tritt dieses Symptom beim Plotten der Graphen auf, so ist höchstwahrscheinlich das Training zu früh beendet worden. 

<img src="Images/tSNE_iterations.png" alt="drawing" width="1000"/>
Quelle: [HYP] 


**Perplexity**, dass zweite wichtige Feature bei der Feineinstellung des t-SNE bestimmt, wie die Gewichtung von lokalen und globalen Eigenschaften der Punkte in unserem Datensatz in die Erstellung der niedrig-dimensionalen Plots mit einfließen soll. Die Auswirkungen verschiedener Werte bei der perplexity sind weitreichend und zugleich nur schwer nachvollziehbar. Während des Trainings wurde hier mit viel Try-And-Error ein idealer Wert versucht anzunähern (hier: 25). Im bebilderten Beispiel wird ein Datensatz mit zwei Clustern und unterschiedlichen Perplexity-Werten (2-100) mittels t-SNE neu berechnet. Im Paper [TODO]: Quelle ["Visualizing Data using t-SNE"](http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf) wird empfohlen, die Perplexity-Werte zwischen fünf und 50 zu initialisieren. Im Beispiel können wir das ursprüngliche Clustering, wenn auch in unterschiedlichen Formen und Intensitäten, gut nachvollziehen. Werte außerhalb dieses empfohlenen Bereichs erzeugen aber gänzlich andere Strukturen. Zu geringe Perplexity-Werte (2) zeigen Cluster mit zu hoher lokaler Dominanz. Bei zu hohen Perplexity-Werten (100) werden die beiden Cluster vereint und können gar nicht mehr unterschieden werden.
Abschließend lässt sich zusammenfassen, dass der Perplexity-Wert eine grobe Richtung vorgibt, wie viele enge Nachbarn ein Punkt unseres Datensatzes hat und individuell für jeden Datensatz ermittelt werden muss.

<img src="Images/tSNE_perplexity.png" alt="drawing" width="1000"/>
Quelle: [HYP]

In [40]:
def compute_tsne_doc(data_df, amount, dimension=2, perplexity=30, learning_rate=200, n_iter=5000):
    '''
    Berechnet t-SNE Embeddings für zufällige Stichprobe aus dem Set der Anfragetexte
    :param data_df: pandas DataFrame mit Spalte 'doc_vector' und 'title'
    :param amount: Größe der Stichprobe
    :param dimension: 2 für 2D oder 3 für 3D
    '''
    tsne = TSNE(perplexity=perplexity, learning_rate=learning_rate, n_components=dimension, init='pca', n_iter=n_iter, method='exact', verbose=1)
    sample = data_df.sample(n=amount)

    doc_vecs = np.array([doc_vec for doc_vec in sample['doc_vector'].values])
    tsne_embeddings = tsne.fit_transform(doc_vecs)

    labels = sample['title'].values
    return tsne_embeddings, labels
    

### Interaktiver 2D/3D Plot mit ploty
Mithilfe von Plotly können wir nun einen ersten 2- oder 3-Dimensionalen Plot unserer Daten die mittels t-SNE errechnet wurden, versuchen.

In [41]:
def plotly_plot_tsne(tsne_embeddings, labels, dimension=3):
    
    marker=dict(
                size=6,
                line=dict(
                    color='rgb(225, 225, 225)',
                    width=0.5
                ),
                opacity=1
            )
    
    if dimension==3:
        x, y, z = zip(*tsne_embeddings)
        
        trace1 = go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers',
            marker=marker,
            text=labels,
            hoverinfo='text'
        )
    else:
        x, y = zip(*tsne_embeddings)
        
        trace1 = go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=marker,
            text=labels,
            hoverinfo='text'
        )
    
    

    data = [trace1]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        xaxis = dict(
            zeroline = False
        ),
        yaxis = dict(
            zeroline = False
        ),
        width=1000,
        height=800,
        #paper_bgcolor= 'rgb(240, 240, 240)',
        #plot_bgcolor= 'rgb(240, 240, 240)'
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

# 3D TSNE Visualisierung für Stichprobe von Anfragen
Im finalen Abschnitt des Notebooks reduzieren wir die Dimensionen einer Stichprobe (Umfang: 200 Dokumente) mittels t-SNE auf 3D und visualisieren diese dann mit Plotly interaktiv.
Reduzierte Stichprobengröße aufgrund von limitierter Hardware, da t-SNE sehr rechenintensiv ist.

**Achtung:** Hier gibt es nur Ergebnisse zu sehen, wenn das Notebook lokal ausgeführt wird, das Modell trainiert und der Datensatz eingeladen wurde. Eine Darstellung auf GitHub selbst ist somit leider nicht möglich.
Auf eine 2D-Darstellung wurde in diesem Fall verzichtet, wäre mit der gleichen Funktion aber auch möglich.

In [42]:
plot_x_docs = 200
tsne_embeddings_3d, labels_3d = compute_tsne_doc(
    data, 
    plot_x_docs, 
    dimension=3,
    perplexity=25,
    learning_rate=10,
    n_iter=1000
)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 0.967027
[t-SNE] KL divergence after 250 iterations with early exaggeration: 46.496586
[t-SNE] KL divergence after 1000 iterations: 0.182286


**Interaktiver 3D Plot der t-SNE Embeddings**  
* Linksklick: Rotieren
* Rechtsklick: Verschieben
* Mausrad: Zoom in/out
* Hover über Punkt: Anzeige des Dokumenten-Titels

Obwohl nur eine Stichprobengröße von 200 gewählt wurde, können lokale Ballungsgebiete, ähnlicher Dokumente gleicher Thematik, identifiziert werden (Hover über Punkt für Titelanzeige).

In [43]:
plotly_plot_tsne(tsne_embeddings_3d, labels_3d, dimension=3)

Der generierte Plot sollte ungefähr folgendermaßen aussehen (statisches Bild des interaktiven Plots zur Darstellung in GitHub):
<img src="Images/plotly_tsne.png" alt="drawing" width="1000"/>

# Literature
<table>
    <tr>
        <td>
            <a id="WED"></a>[WED]
        </td>
        <td>
The Illustrated Word2vec – Jay Alammar – Visualizing machine ...." 27 März. 2019, http://jalammar.github.io/illustrated-word2vec/. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="SAM"></a>[SAM]
        </td>
        <td>
"Word2Vec Tutorial - The Skip-Gram Model · Chris McCormick." 19 Apr.. 2016, http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="SWS"></a>[SWS]
        </td>
        <td>
"The Illustrated Word2vec – Jay Alammar – Visualizing machine ...." 27 März. 2019, http://jalammar.github.io/illustrated-word2vec/. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="NCE"></a>[NCE]
        </td>
        <td>
"Notes on Noise Contrastive Estimation and Negative Sampling." http://demo.clab.cs.cmu.edu/cdyer/nce_notes.pdf. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="CSM"></a>[CSM]
        </td>
        <td>
"Cosine similarity - Wikipedia." https://en.wikipedia.org/wiki/Cosine_similarity. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="TSN"></a>[TSN]
        </td>
        <td>
"Visualizing data using t-SNE - SlideShare." https://www.slideshare.net/KyeongUkJang/visualizing-data-using-tsne-149111155. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="TDS"></a>[TDS]
        </td>
        <td>
"Visualizing Data using t-SNE - Journal of Machine Learning Research." http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>
<table>
    <tr>
        <td>
            <a id="HYP"></a>[HYP]
        </td>
        <td>
"How to Use t-SNE Effectively - Distill.pub." 13 Okt.. 2016, https://distill.pub/2016/misread-tsne/. Aufgerufen am 22 Juni. 2019.
        </td>
    </tr>
</table>




